In [8]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.regularizers import l1_l2
from keras.layers.advanced_activations import LeakyReLU

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import os


In [3]:
#mnistデータのロード
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
#28*28の画像データを1次元の配列に変換
X_train = X_train.reshape(60000, 28*28) / 255
X_test = X_test.reshape(10000, 28*28) / 255

In [5]:
def plot_generated(noise, generator_model, examples=9, plot_dim=(3,3), size=(7,7), epoch=None):
    # noiseからgeneratorで画像を生成
    generated_images = generator_model.predict(noise)

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = generated_images[i, :]
        img = img.reshape((28, 28))
        plt.tight_layout()
        plt.imshow(img, cmap="gray")
        plt.axis("off")
    plt.savefig(os.path.join("generated_figures", str(epoch) + ".png"))
    plt.close()

In [6]:
def plot_metrics(metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

In [10]:
z_input_size = 100
leaky_relu = LeakyReLU()

generator = Sequential([
            Dense(input_dim=z_input_size, units=256, kernel_regularizer=l1_l2(1e-5, 1e-5)),
            BatchNormalization(mode=0),
            leaky_relu,
            Dense(units=512, kernel_regularizer=l1_l2(1e-5, 1e-5)),
            BatchNormalization(mode=0),
            leaky_relu,
            Dense(units=28*28, kernel_regularizer=l1_l2(1e-5, 1e-5)),
            BatchNormalization(mode=0),
            Activation("sigmoid")
            ])
generator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

discriminator = Sequential([
                Dense(input_dim=784, units=1024, kernel_regularizer=l1_l2(1e-5, 1e-5)),
                leaky_relu,
                Dense(units=512, kernel_regularizer=l1_l2(1e-5, 1e-5)),
                leaky_relu,
                Dense(units=256, kernel_regularizer=l1_l2(1e-5, 1e-5)),
                leaky_relu,
                Dense(units=1, kernel_regularizer=l1_l2(1e-5, 1e-5)),   
                Activation("sigmoid")
                ])
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


c:\users\g031o164\appdata\local\continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization()`
  
c:\users\g031o164\appdata\local\continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization()`
  if __name__ == '__main__':
c:\users\g031o164\appdata\local\continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization()`
  if sys.path[0] == '':


Instructions for updating:
Colocations handled automatically by placer.


In [11]:
discriminator.trainable=False
gan = Sequential([
                generator,
                discriminator
                ])
gan.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
discriminator.trainable=True

In [12]:
visualize_train = True
def train(g_model,d_model,gan_model,loss_dict, X_train):
    with tqdm(total=epoch) as pbar:
        for e in range(epoch):
            pbar.update(1)

            # 生成データをノイズから生成
            noise = np.random.uniform(0, 1, size=[batch, z_input_size])
            generated_images = g_model.predict_on_batch(noise)

            # 訓練データをMNISTデータ群から抜粋
            rand_train_index = np.random.randint(0, X_train.shape[0], size=batch)
            image_batch = X_train[rand_train_index, :]

            # 訓練データと生成データを結合
            X = np.vstack((image_batch, generated_images))
            # ラベル作成
            y = np.zeros(int(2*batch))
            y[batch:] = 1
            y = y.astype(int)

            # discriminatorの学習
            d_model.trainable = True
            d_loss = d_model.train_on_batch(x=X, y=y)
            d_model.trainable = False

            # generatorの学習
            noise = np.random.uniform(0, 1, size=[batch, z_input_size])
            y = np.zeros(batch)
            y = y.astype(int)
            g_loss = gan_model.train_on_batch(x=noise, y=y)

            loss_dict["d"].append(d_loss)
            loss_dict["g"].append(g_loss)

            # グラフ描画
            if e%plot_freq == plot_freq-1:
                plot_metrics(loss_dict, int(e/plot_freq))
                
            # 訓練したgeneratorによる生成画像を可視化
            if visualize_train and e < epoch:
                if e%z_plot_freq == z_plot_freq-1:
                    plot_generated(z_group, generator_model=g_model, epoch=int(e/z_plot_freq))
                    #generated_figures.append(fig)

In [14]:
gan_losses = {"d":[], "g":[], "f":[]}
epoch = 100000 #エポック
batch = 1000 #バッチサイズ
z_plot_freq = 1000 #生成画像を保存する間隔
plot_freq = 1000 #lossの画像の保存する間隔
n_train_samples = 60000 #データセットの大きさ
examples = 9 #生成画像の出力枚数

z_group_matrix = np.random.uniform(0,1,examples*z_input_size)
z_group_matrix = z_group_matrix.reshape([9, z_input_size])
z_group = z_group_matrix
print(z_group_matrix.shape)

generated_figures = []

(9, 100)


In [15]:
train(generator,discriminator,gan,loss_dict=gan_losses, X_train=X_train,)

  0%|                                               | 0/100000 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


100%|████████████████████████████████| 100000/100000 [8:30:58<00:00,  3.33it/s]


In [37]:
examples = 9
for i in range(10):
    z_group_matrix = np.random.uniform(0,1,examples*z_input_size)
    z_group_matrix = z_group_matrix.reshape([examples, z_input_size])
    z_group = z_group_matrix
    print(z_group.shape)
    plot_generated(z_group, generator_model=generator, epoch="test"+str(i))

(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
(9, 100)
